<a href="https://colab.research.google.com/github/UiinKim/UiinKim/blob/main/BERT(Bidirectional_Encoder_Representations_from_Transformer).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
from transformers import BertTokenizer

tokenizer=BertTokenizer.from_pretrained("bert-base-uncased") #Bert-base의 토크나이저

In [3]:
result=tokenizer.tokenize('Here is the sentence I want embeddings for.')
#서브워드 토크나이저로 변하는 방식. ##을 붙여서 분리하여 추가한다.
print(result)

['here', 'is', 'the', 'sentence', 'i', 'want', 'em', '##bed', '##ding', '##s', 'for', '.']


In [4]:
print(tokenizer.vocab['here'])

2182


In [5]:
#print(tokenizer.vocab['embeddings'])

In [6]:
print(tokenizer.vocab['em'])

7861


In [7]:
print(tokenizer.vocab['##bed'])

8270


In [8]:
print(tokenizer.vocab['##ing'])

2075


In [9]:
print(tokenizer.vocab['##s'])

2015


In [10]:
#BERT의 단어 집합을 vocabulary.txt에 저장
with open('vocabulary.txt', 'w') as f:
  for token in tokenizer.vocab.keys():
    f.write(token+'\n')

In [11]:
df=pd.read_fwf('vocabulary.txt', header=None)
df

,0
0,[PAD]
1,[unused0]
2,[unused1]
3,[unused2]
4,[unused3]
...,...
30517,##．
30518,##／
30519,##：
30520,##？


In [12]:
print('단어 집합의 크기 : ', len(df))

단어 집합의 크기 :  30522


In [13]:
df.loc[4667].values[0]

'##ding'

In [14]:
df.loc[102].values[0]

'[SEP]'

구글 BERT 마스크드 언어 모델

In [15]:
from transformers import TFBertForMaskedLM
from transformers import AutoTokenizer

In [16]:
model=TFBertForMaskedLM.from_pretrained('bert-large-uncased')
#[MASK]라고 되어있는 단어를 맞추기 위한 마스크드 언어 모델링을 위한 구조로 BERT를 로드
tokenizer=AutoTokenizer.from_pretrained('bert-large-uncased')
#모델 이름을 넣으면 해당 모델이 학습되었을 당시에 사용되었던 토크나이저를 로드

All PyTorch model weights were used when initializing TFBertForMaskedLM.

All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [18]:
inputs=tokenizer('Soccer is a really fun [MASK].', return_tensors='tf')

In [19]:
print(inputs['input_ids'])
#정수 인코딩 확인 결과

tf.Tensor([[ 101 4715 2003 1037 2428 4569  103 1012  102]], shape=(1, 9), dtype=int32)


In [20]:
print(inputs['token_type_ids'])
#문장을 구분하는 세그먼트 인코딩 결과->문장이 2개이고 두번째 문장이 시작되는 구간부터는 1의 시퀀스가 나온다.

tf.Tensor([[0 0 0 0 0 0 0 0 0]], shape=(1, 9), dtype=int32)


In [21]:
print(inputs['attention_mask'])
#실제 단어와 패딩 토큰을 구분하는 어텐션 마스크 -> 패딩이 시작되는 구간부터는 0이 나온다.

tf.Tensor([[1 1 1 1 1 1 1 1 1]], shape=(1, 9), dtype=int32)


In [24]:
from transformers import FillMaskPipeline
pip = FillMaskPipeline(model=model, tokenizer=tokenizer)

In [26]:
pip('Soccer is a really fun [MASK].')
#[MASK]에 들어갈 수 있는 상위 5개의 후보 단어

[{'score': 0.7621126770973206,
  'token': 4368,
  'token_str': 'sport',
  'sequence': 'soccer is a really fun sport.'},
 {'score': 0.20341919362545013,
  'token': 2208,
  'token_str': 'game',
  'sequence': 'soccer is a really fun game.'},
 {'score': 0.012208538129925728,
  'token': 2518,
  'token_str': 'thing',
  'sequence': 'soccer is a really fun thing.'},
 {'score': 0.0018630228005349636,
  'token': 4023,
  'token_str': 'activity',
  'sequence': 'soccer is a really fun activity.'},
 {'score': 0.001335486420430243,
  'token': 2492,
  'token_str': 'field',
  'sequence': 'soccer is a really fun field.'}]

In [28]:
pip('The Avengers is a really fun [MASK].')

[{'score': 0.2562899589538574,
  'token': 2265,
  'token_str': 'show',
  'sequence': 'the avengers is a really fun show.'},
 {'score': 0.17284104228019714,
  'token': 3185,
  'token_str': 'movie',
  'sequence': 'the avengers is a really fun movie.'},
 {'score': 0.11107705533504486,
  'token': 2466,
  'token_str': 'story',
  'sequence': 'the avengers is a really fun story.'},
 {'score': 0.07248983532190323,
  'token': 2186,
  'token_str': 'series',
  'sequence': 'the avengers is a really fun series.'},
 {'score': 0.07046636939048767,
  'token': 2143,
  'token_str': 'film',
  'sequence': 'the avengers is a really fun film.'}]

In [29]:
pip('I went to [MASK] this morning.')

[{'score': 0.35730698704719543,
  'token': 2147,
  'token_str': 'work',
  'sequence': 'i went to work this morning.'},
 {'score': 0.2330448478460312,
  'token': 2793,
  'token_str': 'bed',
  'sequence': 'i went to bed this morning.'},
 {'score': 0.1284504383802414,
  'token': 2082,
  'token_str': 'school',
  'sequence': 'i went to school this morning.'},
 {'score': 0.062305789440870285,
  'token': 3637,
  'token_str': 'sleep',
  'sequence': 'i went to sleep this morning.'},
 {'score': 0.04695258289575577,
  'token': 2465,
  'token_str': 'class',
  'sequence': 'i went to class this morning.'}]

한국어 BERT 마스크드 모델

In [31]:
from transformers import TFBertForMaskedLM
from transformers import AutoTokenizer

In [34]:
#klue/bert-base는 대표적인 한국어 BERT, from_pt=True는 파이토치의 학습 모델을 텐서플로우로 사용한다는 뜻
model=TFBertForMaskedLM.from_pretrained('klue/bert-base', from_pt=True)
tokenizer=AutoTokenizer.from_pretrained("klue/bert-base")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForMaskedLM: ['bert.embeddings.position_ids', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing TFBertForMaskedLM from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForMaskedLM from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [36]:
inputs=tokenizer("축구는 정말 재미있는 [MASK]다.", return_tensors='tf')

In [38]:
#정수 인코딩 결과
print(inputs['input_ids'])

tf.Tensor([[   2 4713 2259 3944 6001 2259    4  809   18    3]], shape=(1, 10), dtype=int32)


In [39]:
#세그먼트 인코딩 결과
print(inputs['token_type_ids'])

tf.Tensor([[0 0 0 0 0 0 0 0 0 0]], shape=(1, 10), dtype=int32)


In [40]:
#실제 단어와 패딩 토큰을 구분하는 attentionmask
print(inputs['attention_mask'])

tf.Tensor([[1 1 1 1 1 1 1 1 1 1]], shape=(1, 10), dtype=int32)


In [41]:
from transformers import FillMaskPipeline
pip=FillMaskPipeline(model=model, tokenizer=tokenizer)

In [42]:
pip("축구는 정말 재미있는 [MASK]다.")

[{'score': 0.8963516354560852,
  'token': 4559,
  'token_str': '스포츠',
  'sequence': '축구는 정말 재미있는 스포츠 다.'},
 {'score': 0.02595745585858822,
  'token': 568,
  'token_str': '거',
  'sequence': '축구는 정말 재미있는 거 다.'},
 {'score': 0.010033913888037205,
  'token': 3682,
  'token_str': '경기',
  'sequence': '축구는 정말 재미있는 경기 다.'},
 {'score': 0.007924334146082401,
  'token': 4713,
  'token_str': '축구',
  'sequence': '축구는 정말 재미있는 축구 다.'},
 {'score': 0.007844174280762672,
  'token': 5845,
  'token_str': '놀이',
  'sequence': '축구는 정말 재미있는 놀이 다.'}]

In [43]:
pip('어벤져스는 정말 재미있는 [MASK]다.')


[{'score': 0.8382413983345032,
  'token': 3771,
  'token_str': '영화',
  'sequence': '어벤져스는 정말 재미있는 영화 다.'},
 {'score': 0.028275731950998306,
  'token': 568,
  'token_str': '거',
  'sequence': '어벤져스는 정말 재미있는 거 다.'},
 {'score': 0.01718936301767826,
  'token': 4665,
  'token_str': '드라마',
  'sequence': '어벤져스는 정말 재미있는 드라마 다.'},
 {'score': 0.014989727176725864,
  'token': 3758,
  'token_str': '이야기',
  'sequence': '어벤져스는 정말 재미있는 이야기 다.'},
 {'score': 0.009382649324834347,
  'token': 4938,
  'token_str': '장소',
  'sequence': '어벤져스는 정말 재미있는 장소 다.'}]

In [44]:
pip('나는 오늘 아침에 [MASK]에 출근을 했다.')


[{'score': 0.08012557774782181,
  'token': 3769,
  'token_str': '회사',
  'sequence': '나는 오늘 아침에 회사 에 출근을 했다.'},
 {'score': 0.06124049797654152,
  'token': 1,
  'token_str': '[UNK]',
  'sequence': '나는 오늘 아침에 에 출근을 했다.'},
 {'score': 0.01748666912317276,
  'token': 4345,
  'token_str': '공장',
  'sequence': '나는 오늘 아침에 공장 에 출근을 했다.'},
 {'score': 0.0161318127065897,
  'token': 5841,
  'token_str': '사무실',
  'sequence': '나는 오늘 아침에 사무실 에 출근을 했다.'},
 {'score': 0.015360800549387932,
  'token': 3671,
  'token_str': '서울',
  'sequence': '나는 오늘 아침에 서울 에 출근을 했다.'}]

Next Sentence Prediction

In [49]:
import tensorflow as tf
from transformers import TFBertForNextSentencePrediction

In [50]:
model=TFBertForNextSentencePrediction.from_pretrained('bert-base-uncased')
tokenizer=AutoTokenizer.from_pretrained('bert-base-uncased')

All PyTorch model weights were used when initializing TFBertForNextSentencePrediction.

All the weights of TFBertForNextSentencePrediction were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForNextSentencePrediction for predictions without further training.


In [51]:
prompt="In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
next_sentence="pizza is eaten with the use of a knife and fork. In casual settings, however, it is cut into wedges to be eaten while held in the hand."

In [52]:
encoding=tokenizer(prompt, next_sentence, return_tensors='tf')

In [53]:
#정수 인코딩 결과
print(encoding['input_ids'])

tf.Tensor(
[[  101  1999  3304  1010 10733  2366  1999  5337 10906  1010  2107  2004
   2012  1037  4825  1010  2003  3591  4895 14540  6610  2094  1012   102
  10733  2003  8828  2007  1996  2224  1997  1037  5442  1998  9292  1012
   1999 10017 10906  1010  2174  1010  2009  2003  3013  2046 17632  2015
   2000  2022  8828  2096  2218  1999  1996  2192  1012   102]], shape=(1, 58), dtype=int32)


In [54]:
#CLS 토큰과 SEP토큰 번호 출력
print(tokenizer.cls_token, ':',tokenizer.cls_token_id)
print(tokenizer.sep_token, ':', tokenizer.sep_token_id)

[CLS] : 101
[SEP] : 102


In [55]:
#디코딩
print(tokenizer.decode(encoding['input_ids'][0]))

[CLS] in italy, pizza served in formal settings, such as at a restaurant, is presented unsliced. [SEP] pizza is eaten with the use of a knife and fork. in casual settings, however, it is cut into wedges to be eaten while held in the hand. [SEP]


In [57]:
#세그먼트 인코딩 결과
print(encoding['token_type_ids'])
#CLS로부터 첫번째 문장이 끝난 뒤로 1로 바뀜

tf.Tensor(
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]], shape=(1, 58), dtype=int32)


3

In [70]:
print(model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0])
print(model(encoding['input_ids'], token_type_ids=encoding['token_type_ids']))



tf.Tensor([[-2.4330919  5.521656 ]], shape=(1, 2), dtype=float32)
TFNextSentencePredictorOutput(loss=None, logits=<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-2.4330919,  5.521656 ]], dtype=float32)>, hidden_states=None, attentions=None)


In [58]:
#model에 입력을 넣고 소프트 맥스 함수 지나기 전의 상태를 저장, 0번 인덱스는 스코어만 빼내는것.
logits=model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]
softmax=tf.keras.layers.Softmax()
#softmax를 통과시킨 후 레이블의 확률값 출력
probs=softmax(logits)
print(probs)

tf.Tensor([[9.9999714e-01 2.8381855e-06]], shape=(1, 2), dtype=float32)


In [60]:
#0번 인덱스의 확률이 1번 인덱스보다 확률이 몇배는 높으므로 가장 확률이 높은 0번 인덱스가 출력이 되었다.-> 두 문장이 실질적으로 이어져있음
print("최종 예측 레이블 : ", tf.math.argmax(probs, axis=-1).numpy())

최종 예측 레이블 :  [0]


In [64]:
#상관 없는 두개의 문장
prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
next_sentence = "The sky is blue due to the shorter wavelength of blue light."
encoding=tokenizer(prompt, next_sentence, return_tensors='tf')

logits=model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax=tf.keras.layers.Softmax()
probs=softmax(logits)
print("최종 예측 레이블 : ", tf.math.argmax(probs, axis=-1).numpy())

최종 예측 레이블 :  [1]


In [65]:
model=TFBertForNextSentencePrediction.from_pretrained('klue/bert-base', from_pt=True)
tokenizer=AutoTokenizer.from_pretrained('klue/bert-base')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForNextSentencePrediction: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForNextSentencePrediction were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForNextSentencePrediction for predictions without further training.


In [66]:
prompt='2002년 월드컵 축구대회는 일본과 공동으로 개최되었떤 세계적은 큰 잔치입니다.'
next_sentence='여행을 가보니 한국의 2002년 월드컵 축구대회의 준비는 완벽했습니다.'
encoding=tokenizer(prompt, next_sentence, return_tensors='tf')

logits=model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax=tf.keras.layers.Softmax()
probs=softmax(logits)
print("최종 예측 레이블 : ", tf.math.argmax(probs, axis=-1).numpy())

최종 예측 레이블 :  [0]


In [67]:
# 상관없는 두 개의 문장
prompt = "2002년 월드컵 축구대회는 일본과 공동으로 개최되었던 세계적인 큰 잔치입니다."
next_sentence = "극장가서 로맨스 영화를 보고싶어요"
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())


최종 예측 레이블 : [1]
